In [6]:
import pandas as pd

try:
    df_2022 = pd.read_csv('ipl_2022_deliveries.csv')
    df_2023 = pd.read_csv('ipl_2023_deliveries.csv')
    df_2024 = pd.read_csv('ipl_2024_deliveries.csv')

    # Merge the dataframes
    merged_df = pd.concat([df_2022, df_2023, df_2024], ignore_index=False)
    print(merged_df)

except FileNotFoundError:
    print("One or more CSV files not found. Please check the file names and paths.")
except Exception as e:
    print(f"An error occurred: {e}")

One or more CSV files not found. Please check the file names and paths.


In [7]:

player_stats = merged_df.groupby('striker')['runs_of_bat'].agg(
    zeroes=lambda x: (x == 0).sum(),
    ones=lambda x: (x == 1).sum(),
    twos=lambda x: (x == 2).sum(),
    threes=lambda x: (x == 3).sum(),
    fours=lambda x: (x == 4).sum(),
    sixes=lambda x: (x == 6).sum(),
    total_runs=lambda x: x.sum()
).reset_index()

# Wickets taken calculation
wickets_df = merged_df[merged_df['wicket_type'].notnull()].groupby('bowler')['wicket_type'].count().reset_index(name='wickets_taken')
player_stats = pd.merge(player_stats, wickets_df, left_on='striker', right_on='bowler', how='left')
player_stats = player_stats.drop(columns='bowler').rename(columns={'batsman': 'player'})
player_stats['wickets_taken'] = player_stats['wickets_taken'].fillna(0).astype(int)

player_stats

NameError: name 'merged_df' is not defined

In [ ]:
try:
    df_ipl_players = pd.read_csv('ipl_2025_players.csv', sep=',')
    print(df_ipl_players)
except FileNotFoundError:
    print("File not found. Please check the file name and path.")
except pd.errors.ParserError:
    print("Error parsing the file. Please check the file format (tab-delimited).")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
try:
    df_ipl_players = df_ipl_players.drop(index=110)  # Drop row with index 109 (110th row)
    print(df_ipl_players)
except NameError:
    print("df_ipl_players not found. Please ensure the dataframe is loaded correctly.")
except IndexError:
    print("Index out of bounds.  Check if row 110 exists in the dataframe.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:

def classify_player(player_name, merged_df):
    total_matches = len(merged_df[(merged_df['bowler'] == player_name) | (merged_df['striker'] == player_name)]['match_id'].unique())
    bowled_matches = len(merged_df[merged_df['bowler'] == player_name]['match_id'].unique())

    if total_matches == 0 :
      return "Not Played"

    ratio = bowled_matches / total_matches
    if ratio == 1:
        return 'Bowler'
    elif ratio >= 0.5:
        return 'All-rounder'
    elif ratio < 0.5:
        return 'Batsman'

player_classifications = []
for player in player_stats['striker']:
    player_classifications.append(classify_player(player, merged_df))

player_stats['Player_Type'] = player_classifications
player_stats

In [ ]:
# Rename 'striker' column to 'player'
player_stats = player_stats.rename(columns={'striker': 'player'})

# Select specific columns
player_stats = player_stats[['player','zeroes', 'ones', 'twos', 'threes', 'fours', 'sixes', 'wickets_taken', 'total_runs', 'Player_Type']]

player_stats

In [ ]:
player_stats['Number of balls played'] = player_stats['zeroes'] + player_stats['ones'] + player_stats['twos'] + player_stats['threes'] + player_stats['fours'] + player_stats['sixes']

# Move the new column to the desired position
cols = list(player_stats.columns)
cols.insert(cols.index('wickets_taken'), cols.pop(cols.index('Number of balls played')))
player_stats = player_stats[cols]

player_stats

In [ ]:
import random
random.seed(0)
# Calculate global distribution
global_distribution = player_stats[['zeroes','ones', 'twos', 'threes', 'fours', 'sixes']].sum()
total_balls_global = global_distribution.sum()
global_distribution = global_distribution / total_balls_global

cutoff = 60

# Function to augment player stats
def augment_stats(row):
    balls_played = row['Number of balls played']
    if balls_played < cutoff:
        remaining_balls = cutoff - balls_played
        # Adjust distribution for bowlers
        if row['Player_Type'] == 'Bowler':
            # Example adjustment: lower probabilities of scoring high runs
            adjusted_global = global_distribution.copy()
            adjusted_global[3] *= 0.2  # Reduce probability of fours
            adjusted_global[4] *= 0.1  # Reduce probability of sixes

            #Ensure probabilities sum to 1
            adjusted_global = adjusted_global / adjusted_global.sum()

            for _ in range(remaining_balls):
                outcome = random.choices(adjusted_global.index, weights=adjusted_global.values, k=1)[0]
                row[outcome] += 1

        else:
            for _ in range(remaining_balls):
                outcome = random.choices(global_distribution.index, weights=global_distribution.values, k=1)[0]
                row[outcome] += 1
    return row

# Apply augmentation
player_stats = player_stats.apply(augment_stats, axis=1)

player_stats

In [ ]:
player_stats['Number of runs'] = (player_stats['ones'] + 2 * player_stats['twos'] + 3 * player_stats['threes'] + 4 * player_stats['fours'] + 6 * player_stats['sixes'])
player_stats['Number of balls played'] = (player_stats['zeroes'] + player_stats['ones'] +  player_stats['twos'] + player_stats['threes'] + player_stats['fours'] + player_stats['sixes'])

# Move the new column to the desired position
cols = list(player_stats.columns)
cols.insert(cols.index('wickets_taken'), cols.pop(cols.index('Number of runs')))
player_stats = player_stats[cols]
player_stats = player_stats.drop(columns=['total_runs'])
player_stats

In [ ]:
# Create a mapping of player names from player_stats to df_ipl_players
name_mapping = {}
for player_name in player_stats['player']:
    for ipl_name in df_ipl_players['Player Name']:
        if player_name in ipl_name:
          name_mapping[ipl_name] = player_name
          break

# Update player names in df_ipl_players based on the mapping
df_ipl_players['Player'] = df_ipl_players['Player Name'].replace(name_mapping)
df_ipl_players

In [ ]:
def arrange_batting_order(team_players):
    team_stats = player_stats[player_stats['player'].isin(team_players)]

    # Sort players based on player type and stats
    batters = team_stats[team_stats['Player_Type'] == 'Batsman'].sort_values(by='Number of runs', ascending=False)
    allrounders = team_stats[team_stats['Player_Type'] == 'All-rounder'].sort_values(by=['Number of runs', 'wickets_taken'], ascending=False)
    bowlers = team_stats[team_stats['Player_Type'] == 'Bowler'].sort_values(by='wickets_taken', ascending=False)

    # Concatenate the sorted groups
    arranged_team = pd.concat([batters, allrounders, bowlers])

    return arranged_team['player'].tolist()

# Example usage (assuming df_ipl_players contains 'Team Name' and 'Player Name')
teams = df_ipl_players['Team'].unique()
for team in teams:
  team_players = df_ipl_players[df_ipl_players['Team'] == team]['Player'].tolist()
  batting_order = arrange_batting_order(team_players)
  print(f"Batting order for {team}: {batting_order}")

In [ ]:
team_batting_orders = {}
for team in teams:
  team_players = df_ipl_players[df_ipl_players['Team'] == team]['Player'].tolist()
  batting_order = arrange_batting_order(team_players)
  team_batting_orders[team] = batting_order

# Accessing batting orders:
# Example:
print(team_batting_orders['KKR'])

In [ ]:
def simulate_match(team1, team2, team_batting_orders, player_stats):
    import random
    random.seed(0)
    # Toss and decide batting order
    toss_winner = random.choice([team1, team2])
    batting_first = toss_winner
    bowling_first = team2 if toss_winner == team1 else team1

    # Initialize scorecard with initial values
    scorecard = {
        batting_first: {
            'runs': 0,
            'wickets': 0,
            'overs': 0,
            'balls': 0,
            'player_scores': {
                team_batting_orders[batting_first][0]: {'runs': 0, 'balls': 0, 'out': False},  # Batsman 1
                team_batting_orders[batting_first][1]: {'runs': 0, 'balls': 0, 'out': False}  # Batsman 2
            },
            'yet_to_bat': team_batting_orders[batting_first][2:],
            'current_batsmen': [team_batting_orders[batting_first][0], team_batting_orders[batting_first][1]],
            'bowler_stats': {}
        },
        bowling_first: {
            'runs': 0,
            'wickets': 0,
            'overs': 0,
            'balls': 0,
            'player_scores': {},
            'yet_to_bat': [],
            'current_batsmen': [],
            'bowler_stats': {}
        }
    }

    def simulate_ball(batsman, bowler):
        batsman_stats = player_stats[player_stats['player'] == batsman].iloc[0]
        bowler_stats = player_stats[player_stats['player'] == bowler].iloc[0]

        total_balls_faced = batsman_stats['Number of balls played']
        outcomes = [0, 1, 2, 3, 4, 6, 'wicket']
        probabilities = [
            batsman_stats['zeroes'] / total_balls_faced,
            batsman_stats['ones'] / total_balls_faced,
            batsman_stats['twos'] / total_balls_faced,
            batsman_stats['threes'] / total_balls_faced,
            batsman_stats['fours'] / total_balls_faced,
            batsman_stats['sixes'] / total_balls_faced,
            bowler_stats['wickets_taken'] / total_balls_faced  # Probability of a wicket
        ]

        # Adjust probabilities to add randomness
        probabilities = [p + random.uniform(-0.1, 0.1) for p in probabilities]
        probabilities = [max(0, p) for p in probabilities]  # Ensure probabilities are non-negative
        probabilities = [p / sum(probabilities) for p in probabilities]  # Normalize probabilities

        # Randomly select an outcome based on probabilities
        outcome = random.choices(outcomes, weights=probabilities)[0]
        return outcome

    def simulate_innings(batting_team, bowling_team, batting_order, target=None):

        # Initialize current_batsmen for the batting team at the start of each innings
        scorecard[batting_team]['current_batsmen'] = [batting_order[0], batting_order[1]]  # Initialize with the first two batsmen
        scorecard[batting_team]['yet_to_bat'] = batting_order[2:]  # Update yet_to_bat
        
        # Initialize player_scores for all batsmen in the order
        for batsman in batting_order:
            scorecard[batting_team]['player_scores'].setdefault(batsman, {'runs': 0, 'balls': 0, 'out': False}) # Initialize before usage

        while scorecard[batting_team]['wickets'] < 10 and scorecard[batting_team]['overs'] < 20:
            bowler = team_batting_orders[bowling_team][-1 - (scorecard[batting_team]['overs'] // 4) % 5]
            scorecard[batting_team]['bowler_stats'].setdefault(bowler, {'overs': 0, 'balls': 0, 'wickets': 0, 'runs_conceded': 0})

            for ball in range(6):
                scorecard[batting_team]['balls'] += 1
                if scorecard[batting_team]['balls'] > 120:
                    break

                batsman = scorecard[batting_team]['current_batsmen'][0]
                non_striker = scorecard[batting_team]['current_batsmen'][1]

                outcome = simulate_ball(batsman, bowler)

                if outcome == 'wicket':
                    scorecard[batting_team]['wickets'] += 1
                    scorecard[batting_team]['player_scores'][batsman]['balls'] += 1
                    scorecard[batting_team]['player_scores'][batsman]['out'] = True

                    scorecard[batting_team]['bowler_stats'][bowler]['wickets'] += 1
                    scorecard[batting_team]['bowler_stats'][bowler]['balls'] += 1

                    if scorecard[batting_team]['wickets'] < 10:
                        next_batsman = scorecard[batting_team]['yet_to_bat'].pop(0) if scorecard[batting_team]['yet_to_bat'] else None
                        if next_batsman:
                            scorecard[batting_team]['player_scores'][next_batsman] = {'runs': 0, 'balls': 0, 'out': False}
                            scorecard[batting_team]['current_batsmen'][0] = next_batsman
                        else:
                            break  # All players are out before wickets are all 10
                    else:
                        break
                else:
                    scorecard[batting_team]['runs'] += outcome
                    scorecard[batting_team]['player_scores'][batsman]['runs'] += outcome
                    scorecard[batting_team]['player_scores'][batsman]['balls'] += 1

                    scorecard[batting_team]['bowler_stats'][bowler]['balls'] += 1
                    scorecard[batting_team]['bowler_stats'][bowler]['runs_conceded'] += outcome

                    # Check if target is reached (for 2nd innings only) - PLACE IT HERE
                    if target is not None and scorecard[batting_team]['runs'] >= target:
                        return  # Stop the innings if target is reached

                    if outcome in (1, 3, 5):  # Rotate strike on odd runs
                        scorecard[batting_team]['current_batsmen'] = scorecard[batting_team]['current_batsmen'][::-1]

                if (ball + 1) % 6 == 0:  # Rotate strike at the end of over
                    scorecard[batting_team]['current_batsmen'] = scorecard[batting_team]['current_batsmen'][::-1]
                    scorecard[batting_team]['overs'] += 1
                    scorecard[batting_team]['bowler_stats'][bowler]['overs'] = scorecard[batting_team]['bowler_stats'][bowler]['balls'] // 6


    def print_scorecard(team):
        print(f"\n{team}: {scorecard[team]['runs']}/{scorecard[team]['wickets']} ({scorecard[team]['overs']}.{scorecard[team]['balls'] % 6} overs)")
        print("Player Scores:")
        for player, stats in scorecard[team]['player_scores'].items():
          if stats['balls'] > 0:  # Only show players who have batted 
            not_out_symbol = "*" if not stats['out'] else ""
            print(f"  {player}: {stats['runs']}{not_out_symbol} ({stats['balls']})")
       
        if scorecard[team]['yet_to_bat']:
            print("\nYet to bat:")
            print(", ".join(scorecard[team]['yet_to_bat']))
        
        # Bowling statistics
        print("\nBowling Statistics:")
        for bowler, stats in scorecard[team if team == bowling_first else batting_first]['bowler_stats'].items():  # Access the bowling team's stats
            print(f"  {bowler}: {stats['overs']}.{stats['balls'] % 6} overs, {stats['wickets']} wickets, {stats['runs_conceded']} runs")

        
        


    print(f"{batting_first} is batting first")
    simulate_innings(batting_first, bowling_first, team_batting_orders[batting_first])
    target = scorecard[batting_first]['runs'] + 1
    simulate_innings(bowling_first, batting_first, team_batting_orders[bowling_first], target=target)

    if scorecard[bowling_first]['runs'] >= target:
        winner = bowling_first
        result = f"{winner} won by {10 - scorecard[bowling_first]['wickets']} wickets"
    elif scorecard[bowling_first]['runs'] == target - 1:
        winner = None
        result = "Match tied"
    else:
        winner = batting_first
        result = f"{winner} won by {target - 1 - scorecard[bowling_first]['runs']} runs"

    print_scorecard(batting_first)
    print_scorecard(bowling_first)
    print(f"\n{result}")

    return scorecard, result
    # Example simulation between KKR and RCB
scorecard, result = simulate_match("KKR", "RCB", team_batting_orders, player_stats)

In [ ]:
try:
    df_matches = pd.read_csv('ipl_2024_matches.csv')
    print(df_matches.head()) # Display first few rows of the dataframe
except FileNotFoundError:
    print("File 'ipl_2024_matches.csv' not found. Please upload the file.")
except pd.errors.ParserError:
    print("Error parsing the CSV file. Please check the file format.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# Keep only specified columns
df_matches = df_matches[['match_no', 'team1', 'team2']]
print(df_matches)

In [3]:
import random
random.seed(0)
def simulate_match_summary(team1, team2, team_batting_orders, player_stats):
    # Toss and decide batting order
    toss_winner = random.choice([team1, team2])
    batting_first = toss_winner
    bowling_first = team2 if toss_winner == team1 else team1

    # Initialize scorecard with initial values
    scorecard = {
        batting_first: {
            'runs': 0,
            'wickets': 0,
            'overs': 0,
            'balls': 0,
            'player_scores': {
                team_batting_orders[batting_first][0]: {'runs': 0, 'balls': 0, 'out': False},  # Batsman 1
                team_batting_orders[batting_first][1]: {'runs': 0, 'balls': 0, 'out': False}  # Batsman 2
            },
            'yet_to_bat': team_batting_orders[batting_first][2:],
            'current_batsmen': [team_batting_orders[batting_first][0], team_batting_orders[batting_first][1]],
            'bowler_stats': {}
        },
        bowling_first: {
            'runs': 0,
            'wickets': 0,
            'overs': 0,
            'balls': 0,
            'player_scores': {},
            'yet_to_bat': [],
            'current_batsmen': [],
            'bowler_stats': {}
        }
    }

    def simulate_ball(batsman, bowler):
        batsman_stats = player_stats[player_stats['player'] == batsman].iloc[0]
        bowler_stats = player_stats[player_stats['player'] == bowler].iloc[0]

        total_balls_faced = batsman_stats['Number of balls played']
        outcomes = [0, 1, 2, 3, 4, 6, 'wicket']
        probabilities = [
            batsman_stats['zeroes'] / total_balls_faced,
            batsman_stats['ones'] / total_balls_faced,
            batsman_stats['twos'] / total_balls_faced,
            batsman_stats['threes'] / total_balls_faced,
            batsman_stats['fours'] / total_balls_faced,
            batsman_stats['sixes'] / total_balls_faced,
            bowler_stats['wickets_taken'] / total_balls_faced  # Probability of a wicket
        ]

        # Adjust probabilities to add randomness
        probabilities = [p + random.uniform(-0.1, 0.1) for p in probabilities]
        probabilities = [max(0, p) for p in probabilities]  # Ensure probabilities are non-negative
        probabilities = [p / sum(probabilities) for p in probabilities]  # Normalize probabilities

        # Randomly select an outcome based on probabilities
        outcome = random.choices(outcomes, weights=probabilities)[0]
        return outcome

    def simulate_innings(batting_team, bowling_team, batting_order, target=None):

        # Initialize current_batsmen for the batting team at the start of each innings
        scorecard[batting_team]['current_batsmen'] = [batting_order[0], batting_order[1]]  # Initialize with the first two batsmen
        scorecard[batting_team]['yet_to_bat'] = batting_order[2:]  # Update yet_to_bat

        # Initialize player_scores for all batsmen in the order
        for batsman in batting_order:
            scorecard[batting_team]['player_scores'].setdefault(batsman, {'runs': 0, 'balls': 0, 'out': False}) # Initialize before usage

        while scorecard[batting_team]['wickets'] < 10 and scorecard[batting_team]['overs'] < 20:
            bowler = team_batting_orders[bowling_team][-1 - (scorecard[batting_team]['overs'] // 4) % 5]
            scorecard[batting_team]['bowler_stats'].setdefault(bowler, {'overs': 0, 'balls': 0, 'wickets': 0, 'runs_conceded': 0})

            for ball in range(6):
                scorecard[batting_team]['balls'] += 1
                if scorecard[batting_team]['balls'] > 120:
                    break

                batsman = scorecard[batting_team]['current_batsmen'][0]
                non_striker = scorecard[batting_team]['current_batsmen'][1]

                outcome = simulate_ball(batsman, bowler)

                if outcome == 'wicket':
                    scorecard[batting_team]['wickets'] += 1
                    scorecard[batting_team]['player_scores'][batsman]['balls'] += 1
                    scorecard[batting_team]['player_scores'][batsman]['out'] = True
                    scorecard[batting_team]['bowler_stats'][bowler]['wickets'] += 1
                    scorecard[batting_team]['bowler_stats'][bowler]['balls'] += 1

                    if scorecard[batting_team]['wickets'] < 10:
                        next_batsman = scorecard[batting_team]['yet_to_bat'].pop(0) if scorecard[batting_team]['yet_to_bat'] else None
                        if next_batsman:
                            scorecard[batting_team]['player_scores'][next_batsman] = {'runs': 0, 'balls': 0, 'out': False}
                            scorecard[batting_team]['current_batsmen'][0] = next_batsman
                        else:
                            break  # All players are out before wickets are all 10
                    else:
                        break  
                else:
                    scorecard[batting_team]['runs'] += outcome
                    scorecard[batting_team]['player_scores'][batsman]['runs'] += outcome
                    scorecard[batting_team]['player_scores'][batsman]['balls'] += 1

                    scorecard[batting_team]['bowler_stats'][bowler]['balls'] += 1
                    scorecard[batting_team]['bowler_stats'][bowler]['runs_conceded'] += outcome

                    # Check if target is reached (for 2nd innings only) - PLACE IT HERE
                    if target is not None and scorecard[batting_team]['runs'] >= target:
                        return  # Stop the innings if target is reached

                    if outcome in (1, 3, 5):  # Rotate strike on odd runs
                        scorecard[batting_team]['current_batsmen'] = scorecard[batting_team]['current_batsmen'][::-1]

                if (ball + 1) % 6 == 0:  # Rotate strike at the end of over
                    scorecard[batting_team]['current_batsmen'] = scorecard[batting_team]['current_batsmen'][::-1]
                    scorecard[batting_team]['overs'] += 1
                    scorecard[batting_team]['bowler_stats'][bowler]['overs'] = scorecard[batting_team]['bowler_stats'][bowler]['balls'] // 6


    def print_scorecard(team):
        print(f"\n{team}: {scorecard[team]['runs']}/{scorecard[team]['wickets']} ({scorecard[team]['overs']}.{scorecard[team]['balls'] % 6} overs)")
    simulate_innings(batting_first, bowling_first, team_batting_orders[batting_first])
    target = scorecard[batting_first]['runs'] + 1
    simulate_innings(bowling_first, batting_first, team_batting_orders[bowling_first], target=target)

    if scorecard[bowling_first]['runs'] >= target:
        winner = bowling_first
        result = f"{winner} won by {10 - scorecard[bowling_first]['wickets']} wickets"
    elif scorecard[bowling_first]['runs'] == target - 1:
        winner = None
        result = "Match tied"
    else:
        winner = batting_first
        result = f"{winner} won by {target - 1 - scorecard[bowling_first]['runs']} runs"

    print_scorecard(batting_first)
    print_scorecard(bowling_first)
    print(f"\n{result}")

    if "won by" in result:
        winning_team = result.split("won by")[0].strip()  # Extract the winner's name
    elif "Match tied" in result:
        winning_team = None  # No winner in case of a tie
    else:
        winning_team = result.split("won by")[0].strip()
    
    return winning_team, scorecard, result

def simulate_season(df_matches, team_batting_orders, player_stats):
    points_table = pd.DataFrame(index=df_ipl_players['Team'].unique(), columns=['Points', 'NRR', 'Total Runs Scored', 'Total Overs Played', 'Total Runs Conceded', 'Total Overs Bowled'])
    points_table.fillna(0, inplace=True)

    for i in range(min(70, len(df_matches))):
        match = df_matches.iloc[i]
        team1 = match['team1']
        team2 = match['team2']

        print(f"\nMATCH {i+1}: {team1} VS {team2}")
        winning_team, scorecard, result = simulate_match_summary(team1, team2, team_batting_orders, player_stats)
        # Award Points
        if "won by" in result:
            winner = result.split("won by")[0].strip()
            points_table.loc[winner, 'Points'] += 2
        elif "Match tied" in result:
            points_table.loc[team1, 'Points'] += 1
            points_table.loc[team2, 'Points'] += 1

          # Update total runs and overs
        points_table.loc[team1, 'Total Runs Scored'] += scorecard[team1]['runs']
        points_table.loc[team1, 'Total Overs Played'] += scorecard[team1]['overs'] + (scorecard[team1]['balls'] % 6) / 10
        points_table.loc[team1, 'Total Runs Conceded'] += scorecard[team2]['runs']
        points_table.loc[team1, 'Total Overs Bowled'] += scorecard[team2]['overs'] + (scorecard[team2]['balls'] % 6) / 10

        points_table.loc[team2, 'Total Runs Scored'] += scorecard[team2]['runs']
        points_table.loc[team2, 'Total Overs Played'] += scorecard[team2]['overs'] + (scorecard[team2]['balls'] % 6) / 10
        points_table.loc[team2, 'Total Runs Conceded'] += scorecard[team1]['runs']
        points_table.loc[team2, 'Total Overs Bowled'] += scorecard[team1]['overs'] + (scorecard[team1]['balls'] % 6) / 10

    # Calculate final NRR for each team after all matches
    points_table['NRR'] = (points_table['Total Runs Scored'] / points_table['Total Overs Played']) - (points_table['Total Runs Conceded'] / points_table['Total Overs Bowled'])
    points_table['NRR'] = points_table['NRR'].fillna(0) # Replace NaN with 0

    # Correct Overs Format
    points_table['Total Overs Played'] = points_table['Total Overs Played'].astype(int) + (points_table['Total Overs Played'] % 1 * 10 / 6).round(1) % 1 * 6 /10
    points_table['Total Overs Bowled'] = points_table['Total Overs Bowled'].astype(int) + (points_table['Total Overs Bowled'] % 1 * 10 / 6).round(1) % 1 * 6 /10

    # Display only Points and NRR columns

    points_table = points_table[['Points', 'NRR']]
    # Sort points table
    points_table = points_table.sort_values(by=['Points', 'NRR'], ascending=[False, False])
    print(points_table)
    return points_table

points_table = simulate_season(df_matches, team_batting_orders, player_stats)

def simulate_playoffs(points_table):
    # Get top 4 teams
    top_4_teams = points_table.head(4).index.tolist()

    playoff_schedule = pd.DataFrame(columns=['Match', 'Team 1', 'Team 2', 'Winner'])

    # Qualifier 1
    team1_qualifier1 = top_4_teams[0]
    team2_qualifier1 = top_4_teams[1]
    winner_qualifier1, scorecard, result = simulate_match_summary(team1_qualifier1, team2_qualifier1, team_batting_orders, player_stats)
    
    # Handle tie in Qualifier 1
    if result == "Match tied":
        winner_qualifier1 = team1_qualifier1  # Higher position in points table wins

    # Eliminator
    team1_eliminator = top_4_teams[2]
    team2_eliminator = top_4_teams[3]
    winner_eliminator, scorecard, result = simulate_match_summary(team1_eliminator, team2_eliminator, team_batting_orders, player_stats)
    
    # Handle tie in Eliminator
    if result == "Match tied":
        winner_eliminator = team1_eliminator  # Higher position in points table wins

    # Qualifier 2
    loser_qualifier1 = team2_qualifier1 if winner_qualifier1 == team1_qualifier1 else team1_qualifier1
    winner_qualifier2, scorecard, result = simulate_match_summary(winner_eliminator, loser_qualifier1, team_batting_orders, player_stats)

    # Handle tie in Qualifier 2
    if result == "Match tied":
        winner_qualifier2 = loser_qualifier1  # Higher position wins between teams who previously had a higher position in pointstable wins

    # Final
    ipl_champion, scorecard, result = simulate_match_summary(winner_qualifier1, winner_qualifier2, team_batting_orders, player_stats)
    if result == "Match tied":
        ipl_champion = winner_qualifier2 if winner_qualifier2 == team1_qualifier1 else winner_qualifier1  # Winner of Qualifier 1 wins in case of tie in the final


    playoff_schedule = pd.DataFrame({
        'Match': ['Qualifier 1', 'Eliminator', 'Qualifier 2', 'Final'],
        'Team 1': [team1_qualifier1, team1_eliminator, winner_eliminator, winner_qualifier1],
        'Team 2': [team2_qualifier1, team2_eliminator, loser_qualifier1, winner_qualifier2],
        'Winner': [winner_qualifier1, winner_eliminator, winner_qualifier2, ipl_champion]
    })

    print(playoff_schedule)
    print(f"\nThe IPL Champions is : {ipl_champion}")

# Assuming 'points_table' is available from the previous code block
simulate_playoffs(points_table)


NameError: name 'df_matches' is not defined